In [0]:
dbutils.widgets.text("proc_date", "")
proc_date = dbutils.widgets.get("proc_date")

In [0]:
%run ../../../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
create_temp_view_with_clean_columns(
    spark,
    catalog_name,
    "udp_wcm_bronze_cx_loyalty",
    "lyt_orders",
    proc_date,
    "temp_cx_loyalty_lyt_orders"
)

In [0]:
spark.sql(f"""
    DELETE FROM {catalog_name}.udp_wcm_silver_cx_loyalty.lyt_orders
    WHERE PROC_DATE = DATE('{proc_date}')
""")

In [0]:
spark.sql(f"""
INSERT INTO {catalog_name}.udp_wcm_silver_cx_loyalty.lyt_orders
SELECT create_date
    ,create_by
    ,update_date
    ,update_by
    ,order_code as order_id
    ,store_no as sale_store_id
    ,store_name as sale_store_name
    ,order_status as order_status_id
    ,concat(channel_id,"-",order_status) as order_channel_status_id
    ,payment_status as order_payment_status_id
    ,concat(channel_id,"-",payment_status) as order_payment_channel_status_id
    ,channel_id as order_chanel_id
    ,'' as order_channel_name
    ,customer_id as order_customer_id
    ,user_id as order_user_id
    ,customer_name as order_customer_name
    ,customer_phone as order_customer_phone
    ,customer_email as order_customer_email
    ,receive_name
    ,receive_phone
    ,receive_email
    ,receive_address
    ,note
    ,bill_to_company as bill_name
    ,bill_to_email as bill_email
    ,bill_to_address as bill_address
    ,tax_code as tax_number
    ,total_net_price as total_bill
    ,total_payment
    ,shipping_fee
    ,discount_amount
    ,redeem_point
    ,payment_method_code as payment_type
    ,receive_province_code
    ,receive_district_code
    ,receive_ward_code
    ,receive_province_name
    ,receive_district_name
    ,receive_ward_name
    ,order_image
    ,vin_id
    ,chain_id
    ,latitude
    ,longitude
    ,seller_portal_ind
    ,employee_refer
    ,proc_date
    ,file_creation_ts
 FROM temp_cx_loyalty_lyt_orders WHERE PROC_DATE = DATE('{proc_date}')
""")